In [53]:
%%writefile mapper.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

stopWords = set(word.strip().lower() for word in open("stop_words_en.txt"))

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
                                                      
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    words = [word.lower() for word in words]
    words = [word for word in words if word not in stopWords]
                                                     
    for word in words:     
        if word.isalpha():
            print "%s\t%s\t%d" % (word, article_id, 1)

Overwriting mapper.py


In [54]:
%%writefile reducer.py

import sys

current_word = None
current_article = None
article_count = 0

for line in sys.stdin:
    try:
        word, article, count = line.strip().split('\t', 2)
    except ValueError as e:
        continue
        
    if current_word != word:
        if current_word:
            print "%s\t%d\t%d" % (current_word, 0, article_count)
        current_word = word
        current_article = article
        article_count = 1
    elif current_article != article:        
        article_count +=1        
        current_article = article

if current_word:
    print "%s\t%d\t%d" % (current_word, 0, article_count)

Overwriting reducer.py


In [55]:
%%writefile mapper1.py

import sys
import re
import os

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

stopWords = set(word.strip().lower() for word in open("stop_words_en.txt"))

for line in sys.stdin:
   
    if "article_count" in os.environ["mapreduce_map_input_file"]:
        print line.strip()
    else:
        try:
            article_id, text = unicode(line.strip()).split('\t', 1)
        except ValueError as e:
            continue
                                                      
        words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
        
        words = [word.lower() for word in words]
        words = [word for word in words if word not in stopWords]
                                                     
        article_size = len(words)
        
        for word in words:
            if word.isalpha():
                print "%s\t%s\t%d" % (word, article_id, article_size)
    


Overwriting mapper1.py


In [56]:
%%writefile reducer1.py

from __future__ import print_function
from math import log
import sys

current_word = None
current_article = None
current_article_size = 0
articles_count = 0
word_count = 0

def printResult():
    tf = 1.0*word_count/current_article_size
    idf = log(4100.0/articles_count)
    print(current_word, current_article, tf*idf, sep="\t")
    return


for line in sys.stdin:
    try:
        word, article, count = line.strip().split('\t', 2)
        count = int(count)
    except ValueError as e:
        continue    
    if article=="0":
        if current_article:
            printResult()
        word_count = 0
        current_article = None
        current_article_size = 0        
        articles_count = count
        current_word = word
    elif current_article != article:
        if current_article:
            printResult()
        current_article = article
        current_article_size = count
        word_count = 1
    else:
        word_count += 1

if current_article:
    printResult()

Overwriting reducer1.py


In [ ]:
%%bash

ARTICLE_COUNT=article_count
OUT=result

NUM_REDUCERS=8

hdfs dfs -rm -r -skipTrash ${ARTICLE_COUNT} > /dev/null
hdfs dfs -rm -r -skipTrash ${OUT} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapreduce.job.name="Streaming word count per article" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
    -D mapreduce.partition.keypartitioner.options="-k1,1" \
    -D mapreduce.partition.keycomparator.options="-k1,2" \
    -D stream.num.map.output.key.fields=2 \
    -files mapper.py,reducer.py,/datasets/stop_words_en.txt \
    -mapper "python mapper.py" \
    -reducer "python reducer.py" \
    -input /data/wiki/en_articles_part \
    -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner \
    -output ${ARTICLE_COUNT} > /dev/null
    
yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapreduce.job.name="Streaming tf-idf" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
    -D mapreduce.partition.keypartitioner.options="-k1,1" \
    -D mapreduce.partition.keycomparator.options="-k1,2" \
    -D stream.num.map.output.key.fields=2 \
    -files mapper1.py,reducer1.py,/datasets/stop_words_en.txt \
    -mapper "python mapper1.py" \
    -reducer "python reducer1.py" \
    -input ${ARTICLE_COUNT},/data/wiki/en_articles_part \
    -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner \
    -output ${OUT} > /dev/null


hdfs dfs -cat result/* |grep -P 'labor\t12\t' | cut -f3